In [ ]:
def bit_flip_err(param):
    from math import sqrt
    from qiskit.quantum_info import Kraus, Pauli, Operator
    
    return Kraus([sqrt(1-param)*Operator(Pauli('I')).data, sqrt(param)*Operator(Pauli('X')).data])

def phase_flip_err(param):
    from math import sqrt
    from qiskit.quantum_info import Kraus, Pauli, Operator
    
    return Kraus([sqrt(1-param)*Operator(Pauli('I')).data, sqrt(param)*Operator(Pauli('Z')).data])

In [ ]:
from math import sqrt
from numpy.random import randint
from qiskit.quantum_info import DensityMatrix, Operator, Pauli

num_parties = [3, 4, 5, 6]

secret = list(randint(2, size = 10000))

Hadamard = Operator([[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]])

num_params = 11
err_list = {}
err_lQEC = {}
for num_party in num_parties:
    print(f'Running for {num_party}-party QSSCM...')
    err_list[num_party] = []
    err_lQEC[num_party] = []
    for param in [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
        print(f'\tRunning for param: {param}...', end = '\r')
        err = 0
        err_QEC = 0
        for s in secret:
            sec = []
            for rep in range(3):
                DM = DensityMatrix([1, 0])

                bob_rand = randint(4)
                match bob_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('X')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                    case 3:
                        DM = DM.evolve(Operator(Pauli('X'))).evolve(Hadamard)
                DM = DM.evolve(bit_flip_err(param)).evolve(phase_flip_err(param))

                charlie_rand = randint(3)
                match charlie_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                DM = DM.evolve(bit_flip_err(param)).evolve(phase_flip_err(param))

                if num_party > 3:
                    dave_rand = randint(3)
                    match dave_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                    DM = DM.evolve(bit_flip_err(param)).evolve(phase_flip_err(param))

                    if num_party > 4:
                        eve_rand = randint(3)
                        match eve_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                        DM = DM.evolve(bit_flip_err(param)).evolve(phase_flip_err(param))

                        if num_party > 5:
                            zach_rand = randint(3)
                            match zach_rand:
                                case 0:
                                    DM = DM.evolve(Operator(Pauli('I')))
                                case 1:
                                    DM = DM.evolve(Operator(Pauli('Y')))
                                case 2:
                                    DM = DM.evolve(Hadamard)
                            DM = DM.evolve(bit_flip_err(param)).evolve(phase_flip_err(param))

                match s:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                DM = DM.evolve(bit_flip_err(param)).evolve(phase_flip_err(param))

                if num_party > 3:
                    if num_party > 4:
                        if num_party > 5:
                            match zach_rand:
                                case 0:
                                    DM = DM.evolve(Operator(Pauli('I')))
                                case 1:
                                    DM = DM.evolve(Operator(Pauli('Y')))
                                case 2:
                                    DM = DM.evolve(Hadamard)

                        match eve_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                    
                    match dave_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)

                match charlie_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)

                match bob_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('X')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                    case 3:
                        DM = DM.evolve(Hadamard).evolve(Operator(Pauli('X')))

                sec.append(int(DM.measure()[0]))
            
            if s != sec[0]:
                err += 1
                
            sec = (sec[0] and sec[1]) or (sec[1] and sec[2]) or (sec[2] and sec[0])
            
            if s != sec:
                err_QEC += 1
                
        err_lQEC[num_party].append(err_QEC/len(secret))
        err_list[num_party].append(err/len(secret))
    print()

print(err_list)
print(err_lQEC)

In [ ]:
err = {}
erq = {}
for num_party in num_parties:
    err[num_party] = []
    erq[num_party] = []
    for param in [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
        err[num_party].append(0.5*(1-(1-2*param)**num_party))
        erq[num_party].append(3*err[num_party][-1]**2*(1-err[num_party][-1])+err[num_party][-1]**3)

print(err)
print(erq)

In [ ]:
class five_qubit_code:
    def __init__(self):
        self.num_physical_qubits = 5    # per logical qubit
        self._get_correction_ops()
        pass
    
    def encode(self, DM, inverse = False):
        '''Encoding for [[5,1,3]] code from [PhysRevLett.77.198]
        Arguments:
            DM [<DensityMatrix>]: Logical state after time evaluation
            inverse [bool]: Switches between encoding and decoding
        Returns:
            Encoded Logical state
        '''

        from math import log2
        from qiskit import QuantumCircuit
        from qiskit.circuit.library import MCMT
        from qiskit.quantum_info import Operator, DensityMatrix

        num_logical_qubits = int(log2(DM.dim)//self.num_physical_qubits)    # Number of Logical qubits (1 or 2)

        # Initialize the circuit
        qc = QuantumCircuit(self.num_physical_qubits*num_logical_qubits)

        if not inverse:
            num_logical_qubits = int(log2(DM.dim))

            # Initialize the circuit
            qc = QuantumCircuit(self.num_physical_qubits*num_logical_qubits)
            
            # Append ancilas
            anc = DensityMatrix([1] + [0]*(2**(~-self.num_physical_qubits*num_logical_qubits)-1))
            DM = DM.expand(anc)

            # Main physical qubit would be in the middle of each logical qubits
            for i in range(num_logical_qubits)[::-1]:
                qc.swap(i, self.num_physical_qubits*i+self.num_physical_qubits//2)

        # Encoding
        for i in range(num_logical_qubits):
            qc.h([1+self.num_physical_qubits*i, 3+self.num_physical_qubits*i, 4+self.num_physical_qubits*i])
            qc.append(MCMT('z', 2, 1), [3+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.x([3+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.append(MCMT('z', 2, 1), [3+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.x([3+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.cx([2+self.num_physical_qubits*i, 4+self.num_physical_qubits*i, 4+self.num_physical_qubits*i, 1+self.num_physical_qubits*i, 3+self.num_physical_qubits*i], [0+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 0+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 0+self.num_physical_qubits*i])
            qc.cz(1+self.num_physical_qubits*i, 0+self.num_physical_qubits*i)
            qc.append(MCMT('z', 2, 1), [3+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.x([3+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.append(MCMT('z', 2, 1), [3+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])
            qc.x([3+self.num_physical_qubits*i, 2+self.num_physical_qubits*i, 1+self.num_physical_qubits*i])

        # Convert the circuit into a Operator
        op = Operator(qc)
        if inverse:
            op = Operator(qc.inverse())
        return DM.evolve(op)
    
    def reconstruct(self, DM):
        '''Reconstructs the logical state correcting errors
        Arguments:
            DM [<DensityMatrix>]: Logical state after time evaluation
        Returns:
            Reconstructed logical state as DensityMatrix
        '''

        from math import log2
        from itertools import product
        from qiskit.quantum_info import Operator, Pauli, DensityMatrix, partial_trace

        num_logical_qubits = int(log2(DM.dim)//self.num_physical_qubits)    # Number of Logical qubits (1 or 2)
        
        # Decode the logical state
        DM = self.encode(DM, True)
        
        # Correction operations
        correction_ops = list(product(self.correction_ops, repeat = num_logical_qubits))
        
        # Apply correction operations
        correction_op = correction_ops[0][0]
        for op in correction_ops[0][1:]:
            correction_op = correction_op.tensor(op)
        rho = DM.evolve(correction_op).data
        for ops in correction_ops[1:]:
            correction_op = ops[0]
            for op in ops[1:]:
                correction_op = correction_op.tensor(op)
            rho += DM.evolve(correction_op).data
        DM = DensityMatrix(rho)
        
        # Remove the ancillas
        for i in range(num_logical_qubits)[::-1]:
            DM = partial_trace(DM, [0+self.num_physical_qubits*i, 1+self.num_physical_qubits*i, 3+self.num_physical_qubits*i, 4+self.num_physical_qubits*i])
        
        # Reconstruct the Logical state and return
        return self.encode(DM)

    def _get_correction_ops(self):
        '''Calculates correction operations for single logical qubit'''
        
        from qiskit.quantum_info import Operator, Pauli
        
        Op = [Operator([[1, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]),
              Operator([[0, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]),
              Operator([[0, 0, 1, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]),
              Operator([[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]])]    # Error detection operations
        
        ops = []
        for i in range(16):
            # Traverse through all possible errors
            c = f'{bin(i)}'[2:]
            r = 4 - len(c)
            for _ in range(r):
                c = '0' + c

            # Get Pauli correction operation
            if i in [0, 2, 3, 4, 8]:
                # No error
                pauli = Pauli('I')
            elif i in [1, 5, 10, 12, 15]:
                # Phase-flip error
                pauli = Pauli('Z')
            elif i in [6, 7, 9, 11, 14]:
                # Bit-flip error
                pauli = Pauli('X')
            else:
                # Bit-phase-flip error
                pauli = Pauli('X').compose(Pauli('Z'))

            # Detect error and get correction operatios
            ops.append(Op[int(c[:2], 2)].tensor(pauli).tensor(Op[int(c[2:], 2)]))
        
        self.correction_ops = ops

In [ ]:
def bit_flip_err(param, num_qubits = 1):
    from math import sqrt
    from qiskit.quantum_info import Kraus, Pauli, Operator
    
    K1 = Kraus([sqrt(1-param)*Operator(Pauli('I')).data, sqrt(param)*Operator(Pauli('X')).data])
    K = Kraus([sqrt(1-param)*Operator(Pauli('I')).data, sqrt(param)*Operator(Pauli('X')).data])
    for _ in range(num_qubits-1):
        K = K.expand(K1)
    return K

def phase_flip_err(param, num_qubits = 1):
    from math import sqrt
    from qiskit.quantum_info import Kraus, Pauli
    
    K1 =  Kraus([sqrt(1-param)*Operator(Pauli('I')).data, sqrt(param)*Operator(Pauli('Z')).data])
    K =  Kraus([sqrt(1-param)*Operator(Pauli('I')).data, sqrt(param)*Operator(Pauli('Z')).data])
    for _ in range(num_qubits-1):
        K = K.expand(K1)
    return K

In [ ]:
# Multi-cycle 5-qubit QEC

from math import sqrt
from numpy.random import randint
from qiskit.quantum_info import DensityMatrix, Operator, Pauli, partial_trace

QEC5 = five_qubit_code()

num_parties = [3, 4, 5, 6][:1]

secret = list(randint(2, size = 10000))

Hadamard = Operator([[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]])

num_params = 11
err_mQEC5 = {}
for num_party in num_parties:
    print(f'Running for {num_party}-party QSSCM...')
    err_mQEC5[num_party] = []
    for param in [0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 1.0]:
        print(f'\tRunning for param: {param}...', end = '\r')
        err = 0
        err_QEC = 0
        for s in secret:
            DM = DensityMatrix([1, 0])

            bob_rand = randint(4)
            match bob_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('X')))
                case 2:
                    DM = DM.evolve(Hadamard)
                case 3:
                    DM = DM.evolve(Operator(Pauli('X'))).evolve(Hadamard)
            DM = QEC5.encode(DM).evolve(bit_flip_err(param, 5)).evolve(phase_flip_err(param, 5))
            DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            charlie_rand = randint(3)
            match charlie_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
                case 2:
                    DM = DM.evolve(Hadamard)
            DM = QEC5.encode(DM).evolve(bit_flip_err(param, 5)).evolve(phase_flip_err(param, 5))
            DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            if num_party > 3:
                dave_rand = randint(3)
                match dave_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                DM = QEC5.encode(DM).evolve(bit_flip_err(param, 5)).evolve(phase_flip_err(param, 5))
                DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

                if num_party > 4:
                    eve_rand = randint(3)
                    match eve_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                    DM = QEC5.encode(DM).evolve(bit_flip_err(param, 5)).evolve(phase_flip_err(param, 5))
                    DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

                    if num_party > 5:
                        zach_rand = randint(3)
                        match zach_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                        DM = QEC5.encode(DM).evolve(bit_flip_err(param, 5)).evolve(phase_flip_err(param, 5))
                        DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            match s:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
            DM = QEC5.encode(DM).evolve(bit_flip_err(param, 5)).evolve(phase_flip_err(param, 5))
            DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            if num_party > 3:
                if num_party > 4:
                    if num_party > 5:
                        match zach_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)

                    match eve_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                
                match dave_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)

            match charlie_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
                case 2:
                    DM = DM.evolve(Hadamard)

            match bob_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('X')))
                case 2:
                    DM = DM.evolve(Hadamard)
                case 3:
                    DM = DM.evolve(Hadamard).evolve(Operator(Pauli('X')))

            sec = int(DM.measure()[0])
            
            if s != sec:
                err_QEC += 1
                
        err_mQEC5[num_party].append(err_QEC/len(secret))
    print()

print(err_mQEC5)

In [ ]:
def damp_err(param):
    from math import sqrt
    from numpy import eye, zeros
    from qiskit.quantum_info import Kraus, Pauli, Operator
    
    E = [eye(2), zeros((2, 2))]
    E[0][1][1] = sqrt(1-param)
    E[1][0][1] = sqrt(param)
    
    return Kraus(E)

In [ ]:
from math import sqrt
from numpy.random import randint
from qiskit.quantum_info import DensityMatrix, Operator, Pauli

num_parties = [3, 4, 5, 6]

secret = list(randint(2, size = 10000))

Hadamard = Operator([[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]])

num_params = 11
damp_err_list = {}
damp_err_lQEC = {}
for num_party in num_parties:
    print(f'Running for {num_party}-party QSSCM...')
    damp_err_list[num_party] = []
    damp_err_lQEC[num_party] = []
    for param in [0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0]:
        print(f'\tRunning for param: {param}...', end = '\r')
        err = 0
        err_QEC = 0
        for s in secret:
            sec = []
            for rep in range(3):
                DM = DensityMatrix([1, 0])

                bob_rand = randint(4)
                match bob_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('X')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                    case 3:
                        DM = DM.evolve(Operator(Pauli('X'))).evolve(Hadamard)
                DM = DM.evolve(damp_err(param))

                charlie_rand = randint(3)
                match charlie_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                DM = DM.evolve(damp_err(param))

                if num_party > 3:
                    dave_rand = randint(3)
                    match dave_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                    DM = DM.evolve(damp_err(param))

                    if num_party > 4:
                        eve_rand = randint(3)
                        match eve_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                        DM = DM.evolve(damp_err(param))

                        if num_party > 5:
                            zach_rand = randint(3)
                            match zach_rand:
                                case 0:
                                    DM = DM.evolve(Operator(Pauli('I')))
                                case 1:
                                    DM = DM.evolve(Operator(Pauli('Y')))
                                case 2:
                                    DM = DM.evolve(Hadamard)
                            DM = DM.evolve(damp_err(param))

                match s:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                DM = DM.evolve(damp_err(param))

                if num_party > 3:
                    if num_party > 4:
                        if num_party > 5:
                            match zach_rand:
                                case 0:
                                    DM = DM.evolve(Operator(Pauli('I')))
                                case 1:
                                    DM = DM.evolve(Operator(Pauli('Y')))
                                case 2:
                                    DM = DM.evolve(Hadamard)

                        match eve_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                    
                    match dave_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)

                match charlie_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)

                match bob_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('X')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                    case 3:
                        DM = DM.evolve(Hadamard).evolve(Operator(Pauli('X')))

                sec.append(int(DM.measure()[0]))
            
            if s != sec[0]:
                err += 1
                
            sec = (sec[0] and sec[1]) or (sec[1] and sec[2]) or (sec[2] and sec[0])
            
            if s != sec:
                err_QEC += 1
                
        damp_err_lQEC[num_party].append(err_QEC/len(secret))
        damp_err_list[num_party].append(err/len(secret))
    print()

print(damp_err_list)
print(damp_err_lQEC)

In [ ]:
def damp_err(param, num_qubits):
    from math import sqrt
    from numpy import eye, zeros
    from qiskit.quantum_info import Kraus, Pauli, Operator
    
    E = [eye(2), zeros((2, 2))]
    E[0][1][1] = sqrt(1-param)
    E[1][0][1] = sqrt(param)
    E = Kraus(E)
    E1 = E.copy()
    
    for _ in range(num_qubits-1):
        E = E.expand(E1)
    
    return Kraus(E)

In [ ]:
# Multi-cycle 5-qubit QEC (Amplitude Damping)

from math import sqrt
from numpy.random import randint
from qiskit.quantum_info import DensityMatrix, Operator, Pauli, partial_trace

QEC5 = five_qubit_code()

num_parties = [3, 4, 5, 6][:1]

secret = list(randint(2, size = 10000))

Hadamard = Operator([[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]])

num_params = 11
err_dQEC5 = {}
for num_party in num_parties:
    print(f'Running for {num_party}-party QSSCM...')
    err_dQEC5[num_party] = []
    for param in [0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.45, 0.5, 0.6, 0.8, 1.0]:
        print(f'\tRunning for param: {param}...', end = '\r')
        err = 0
        err_QEC = 0
        for s in secret:
            DM = DensityMatrix([1, 0])

            bob_rand = randint(4)
            match bob_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('X')))
                case 2:
                    DM = DM.evolve(Hadamard)
                case 3:
                    DM = DM.evolve(Operator(Pauli('X'))).evolve(Hadamard)
            DM = QEC5.encode(DM).evolve(damp_err(param, 5))
            DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            charlie_rand = randint(3)
            match charlie_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
                case 2:
                    DM = DM.evolve(Hadamard)
            DM = QEC5.encode(DM).evolve(damp_err(param, 5))
            DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            if num_party > 3:
                dave_rand = randint(3)
                match dave_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                DM = QEC5.encode(DM).evolve(damp_err(param, 5))
                DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

                if num_party > 4:
                    eve_rand = randint(3)
                    match eve_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                    DM = QEC5.encode(DM).evolve(damp_err(param, 5))
                    DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

                    if num_party > 5:
                        zach_rand = randint(3)
                        match zach_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                        DM = QEC5.encode(DM).evolve(damp_err(param, 5))
                        DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            match s:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
            DM = QEC5.encode(DM).evolve(damp_err(param, 5))
            DM = partial_trace(QEC5.encode(QEC5.reconstruct(DM), True), [0, 1, 3, 4])

            if num_party > 3:
                if num_party > 4:
                    if num_party > 5:
                        match zach_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)

                    match eve_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                
                match dave_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)

            match charlie_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
                case 2:
                    DM = DM.evolve(Hadamard)

            match bob_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('X')))
                case 2:
                    DM = DM.evolve(Hadamard)
                case 3:
                    DM = DM.evolve(Hadamard).evolve(Operator(Pauli('X')))

            sec = int(DM.measure()[0])
            
            if s != sec:
                err_QEC += 1
                
        err_dQEC5[num_party].append(err_QEC/len(secret))
    print()

print(err_dQEC5)

In [ ]:
class four_qubit_code:
    def __init__(self):
        self.num_qubit = 4
        self.enc_op = self._get_enc_op()    #Encoding operator
        self.synd_op = self._get_synd_op()    #Syndrome measurement operator
        self.cor_op = None    #Correction operator
        pass
    
    def _get_enc_op(self):
        '''Creates Encoding operator'''
        
        from qiskit import QuantumCircuit
        from qiskit.quantum_info import Operator
        
        # Create corresponding quantum circuit
        qc = QuantumCircuit(self.num_qubit)
        
        # Encode operations
        qc.h(0)
        qc.cx([3, 0, 0, 0], [2, 1, 2, 3])
        
        return Operator(qc)
    
    def _get_synd_op(self):
        '''Creates syndrome measurement operation'''
        
        from qiskit import QuantumCircuit
        from qiskit.quantum_info import Operator
        
        # Create corresponding quantum circuit
        qc = QuantumCircuit(self.num_qubit)
        
        # Apply operations
        qc.cx([0, 3], [1, 2])
        
        return Operator(qc)
    
    def get_cor_op(self, gamma):
        '''Creates correction operation
        gamma [float]: Damping probability
        '''
        
        from numpy import arctan, arccos, pi
        from qiskit import QuantumCircuit
        from qiskit.quantum_info import Operator
        
        self.cor_op = []
        for i in range(3):
            if not i:
                theta = arctan((1-gamma)**2)
                qc = QuantumCircuit(2)
                qc.cx(1, 0)
                qc.ry(2*theta, 1)
                qc.cry(pi/2-2*theta, 0, 1)
                self.cor_op.append(Operator(qc))
            elif not ~-i:
                theta = arccos(1-gamma)
                qc = QuantumCircuit(3)
                qc.x(2)
                qc.cry(theta, 2, 0)
                self.cor_op.append(Operator(qc))
            elif not ~-~-i:
                theta = arccos(1-gamma)
                qc = QuantumCircuit(3)
                qc.x(1)
                qc.cry(theta, 1, 0)
                self.cor_op.append(Operator(qc))
        pass
    
    def encode(self, DM):
        '''Encodes physical qubit to logical qubit
        DM [<DensityMatrix>]: Physical state as DensityMatrix
        
        Returns: Logical state as DensityMatrix
        '''
        
        anc = [1] + [0] * ~-2**(~-self.num_qubit)
        ancilla = DensityMatrix(anc)
        state_exp = ancilla.expand(DM)
        return state_exp.evolve(self.enc_op)
    
    def correct(self, DM):
        '''Corrects logical qubit and returns corresponding physical qubit
        DM [<DensityMatrix>]: Logical state as DensityMatrix
        
        Returns: Physical state as DensityMatrix
        '''
        from qiskit.quantum_info import partial_trace
        
        # Syndrome measurement
        synd_res, DM = DM.evolve(self.synd_op).measure([1, 2])
        DM = partial_trace(DM, [1, 2])
        
        # Apply correction operation
        if not int(synd_res, 2):
            DM = DM.evolve(self.cor_op[int(synd_res, 2)])
            DM = partial_trace(DM.measure([1])[1], [1])
        elif not ~-int(synd_res, 2):
            DM = DensityMatrix([1, 0]).expand(DM).evolve(self.cor_op[int(synd_res, 2)])
            DM = partial_trace(DM.measure([0])[1], [0, 1])
        elif not ~-~-int(synd_res, 2):
            DM = DensityMatrix([1, 0]).expand(DM).evolve(self.cor_op[int(synd_res, 2)])
            DM = partial_trace(DM.measure([0])[1], [0, 2])
        else:
            DM = partial_trace(DM.measure([1])[1], [1])
		
        return DM

In [ ]:
# Multi-cycle 4-qubit QEC (Amplitude Damping)

from math import sqrt
from numpy.random import randint
from qiskit.quantum_info import DensityMatrix, Operator, Pauli

QEC4 = four_qubit_code()

num_parties = [3, 4, 5, 6][:1]

secret = list(randint(2, size = 10000))

Hadamard = Operator([[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]])

num_params = 11
err_dQEC4 = {}
for num_party in num_parties:
    print(f'Running for {num_party}-party QSSCM...')
    err_dQEC4[num_party] = []
    for param in [0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0]:
        print(f'\tRunning for param: {param}...', end = '\r')
        QEC4.get_cor_op(param)
        err = 0
        err_QEC = 0
        for s in secret:
            DM = DensityMatrix([1, 0])

            bob_rand = randint(4)
            match bob_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('X')))
                case 2:
                    DM = DM.evolve(Hadamard)
                case 3:
                    DM = DM.evolve(Operator(Pauli('X'))).evolve(Hadamard)
            DM = QEC4.encode(DM).evolve(damp_err(param, 4))
            DM = QEC4.correct(DM)

            charlie_rand = randint(3)
            match charlie_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
                case 2:
                    DM = DM.evolve(Hadamard)
            DM = QEC4.encode(DM).evolve(damp_err(param, 4))
            DM = QEC4.correct(DM)

            if num_party > 3:
                dave_rand = randint(3)
                match dave_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)
                DM = QEC4.encode(DM).evolve(damp_err(param, 4))
                DM = QEC4.correct(DM)

                if num_party > 4:
                    eve_rand = randint(3)
                    match eve_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                    DM = QEC4.encode(DM).evolve(damp_err(param, 4))
                    DM = QEC4.correct(DM)

                    if num_party > 5:
                        zach_rand = randint(3)
                        match zach_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)
                        DM = QEC4.encode(DM).evolve(damp_err(param, 4))
                        DM = QEC4.correct(DM)

            match s:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
            DM = QEC4.encode(DM).evolve(damp_err(param, 4))
            DM = QEC4.correct(DM)

            if num_party > 3:
                if num_party > 4:
                    if num_party > 5:
                        match zach_rand:
                            case 0:
                                DM = DM.evolve(Operator(Pauli('I')))
                            case 1:
                                DM = DM.evolve(Operator(Pauli('Y')))
                            case 2:
                                DM = DM.evolve(Hadamard)

                    match eve_rand:
                        case 0:
                            DM = DM.evolve(Operator(Pauli('I')))
                        case 1:
                            DM = DM.evolve(Operator(Pauli('Y')))
                        case 2:
                            DM = DM.evolve(Hadamard)
                
                match dave_rand:
                    case 0:
                        DM = DM.evolve(Operator(Pauli('I')))
                    case 1:
                        DM = DM.evolve(Operator(Pauli('Y')))
                    case 2:
                        DM = DM.evolve(Hadamard)

            match charlie_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('Y')))
                case 2:
                    DM = DM.evolve(Hadamard)

            match bob_rand:
                case 0:
                    DM = DM.evolve(Operator(Pauli('I')))
                case 1:
                    DM = DM.evolve(Operator(Pauli('X')))
                case 2:
                    DM = DM.evolve(Hadamard)
                case 3:
                    DM = DM.evolve(Hadamard).evolve(Operator(Pauli('X')))

            sec = int(DM.measure()[0])
            
            if s != sec:
                err_QEC += 1
                
        err_dQEC4[num_party].append(err_QEC/len(secret))
    print()

print(err_dQEC4)

In [ ]:
from matplotlib.pyplot import rcParams, tick_params
from matplotlib import rc

SIZE = 25
rc('font', size = SIZE)
rc('axes', titlesize=SIZE)
rcParams["figure.figsize"] = (13, 10)
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Arial'] + rcParams['font.serif']

In [ ]:
from matplotlib.pyplot import subplots, legend, show, savefig, tick_params

_, ax = subplots(1, 1)
ax.plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_list[3], label = 'No encoding', color = '#a00')
ax.plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_lQEC[3], label = 'Repetition Code', ls = '--', color = '#00a')
ax.plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], err_dQEC5[3], label = '[[5,1,3]] perfect QEC', ls = '-.', color = '#a0a')
ax.plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], err_dQEC4[3], label = '[[4,1]] approximate QEC', ls = ':', color = '#000')


ax.set_xlabel(r'Channel damping strength $\gamma$')
ax.set_ylabel(r'Error on secret')

legend()

tick_params(axis = 'both', which = 'both', direction = 'in')

savefig("QEC_amp.png", format="png", bbox_inches="tight")

show()

In [ ]:
from matplotlib.pyplot import subplots, legend, show, savefig, tick_params

phys_err = [i/100 for i in range(101)]

_, ax = subplots(1, 1)
ax.plot(phys_err, [0.5*(1-(1-2*p)**3) for p in phys_err], label = r'$n = 3$', color = '#a00')
ax.plot(phys_err, [0.5*(1-(1-2*p)**4) for p in phys_err], label = r'$n = 4$', ls = '--', color = '#00a')
ax.plot(phys_err, [0.5*(1-(1-2*p)**5) for p in phys_err], label = r'$n = 5$', ls = '-.', color = '#a0a')
ax.plot(phys_err, [0.5*(1-(1-2*p)**6) for p in phys_err], label = r'$n = 6$', ls = ':', color = '#000')


ax.set_xlabel(r'Qubit error $p$')
ax.set_ylabel(r'Error on secret $e_1^g$')

legend()

tick_params(axis = 'both', which = 'both', direction = 'in')

savefig("err_flip_gen.png", format="png", bbox_inches="tight")

show()

In [ ]:
from matplotlib.pyplot import subplots, legend, show, savefig, tick_params

_, ax = subplots(1, 1)
ax.plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_list[3], label = 'No encoding', color = '#a00')
ax.plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_lQEC[3], label = 'Repetition Code', ls = '--', color = '#00a')
ax.plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.85, 0.9, 0.95, 1.0], err_mQEC5[3], label = '[[5,1,3]] QEC', ls = '-.', color = '#a0a')


ax.set_xlabel(r'Channel error probability $p$')
ax.set_ylabel(r'Error on secret')

legend()

tick_params(axis = 'both', which = 'both', direction = 'in')

savefig("QEC_flip.png", format="png", bbox_inches="tight")

show()

In [ ]:
from matplotlib.pyplot import rcParams, tick_params
from matplotlib import rc

SIZE = 10
rc('font', size = SIZE)
rc('axes', titlesize=SIZE)
rcParams["figure.figsize"] = (13, 10)
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Arial'] + rcParams['font.serif']

In [ ]:
from matplotlib.pyplot import subplots, legend, show, savefig, subplots_adjust, tick_params

fig, ax = subplots(2, 2)

subplots_adjust(hspace = 0.3)

ax[0, 0].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err[3], label = r'No encoding $e_1$, analytic', color = '#a00')
ax[0, 0].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], erq[3], label = r'Repetition code $e_f$, analytic', ls = '--', color = '#00a')
ax[0, 0].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_list[3], label = r'No encoding $e_1$, simulation', marker = 'o', color = '#a00')
ax[0, 0].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_lQEC[3], label = r'Repetition code $e_f$, simulation', marker = '^', color = '#00a')

ax[0, 0].set_xlabel(r'Channel error probability $p$')
ax[0, 0].set_ylabel(r'Error on secret $e_1$ or $e_f$')

ax[0, 0].legend()

ax[0, 0].set_title('(a) 3-party QSSCM')

ax[0, 1].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err[4], label = r'No encoding $e_1$, analytic', color = '#a00')
ax[0, 1].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], erq[4], label = r'Repetition code $e_f$, analytic', ls = '--', color = '#00a')
ax[0, 1].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_list[4], label = r'No encoding $e_1$, simulation', marker = 'o', color = '#a00')
ax[0, 1].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_lQEC[4], label = r'Repetition code $e_f$, simulation', marker = '^', color = '#00a')

ax[0, 1].set_xlabel(r'Channel error probability $p$')

ax[0, 1].legend()

ax[0, 1].set_title('(b) 4-party QSSCM')

ax[1, 0].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err[5], label = r'No encoding $e_1$, analytic', color = '#a00')
ax[1, 0].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], erq[5], label = r'Repetition code $e_f$, analytic', ls = '--', color = '#00a')
ax[1, 0].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_list[5], label = r'No encoding $e_1$, simulation', marker = 'o', color = '#a00')
ax[1, 0].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_lQEC[5], label = r'Repetition code $e_f$, simulation', marker = '^', color = '#00a')

ax[1, 0].set_xlabel(r'Channel error probability $p$')
ax[1, 0].set_ylabel(r'Error on secret $e_1$ or $e_f$')

ax[1, 0].legend()

ax[1, 0].set_title('(c) 5-party QSSCM')

ax[1, 1].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err[6], label = r'No encoding $e_1$, analytic', color = '#a00')
ax[1, 1].plot([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], erq[6], label = r'Repetition code $e_f$, analytic', ls = '--', color = '#00a')
ax[1, 1].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_list[6], label = r'No encoding $e_1$, simulation', marker = 'o', color = '#a00')
ax[1, 1].scatter([0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1.0], err_lQEC[6], label = r'Repetition code $e_f$, simulation', marker = '^', color = '#00a')

ax[1, 1].set_xlabel(r'Channel error probability $p$')

ax[1, 1].legend()

ax[1, 1].set_title('(d) 6-party QSSCM')

tick_params(axis = 'both', which = 'both', direction = 'in')

savefig("rep_flip.png", format="png", bbox_inches="tight")

show()

In [ ]:
from matplotlib.pyplot import subplots, legend, show, savefig, subplots_adjust, tick_params

fig, ax = subplots(2, 2)

subplots_adjust(hspace = 0.3)

ax[0, 0].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_list[3], label = r'No encoding $e_1$', color = '#a00')
ax[0, 0].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_lQEC[3], label = r'Repetition code $e_f$', color = '#00a', ls = '--')

ax[0, 0].set_xlabel(r'Channel damping strength $\gamma$')
ax[0, 0].set_ylabel(r'Error on secret $e_1$ or $e_f$')

ax[0, 0].legend()

ax[0, 0].set_title('(a) 3-party QSSCM')

ax[0, 1].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_list[4], label = r'No encoding $e_1$', color = '#a00')
ax[0, 1].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_lQEC[4], label = r'Repetition code $e_f$', color = '#00a', ls = '--')

ax[0, 1].set_xlabel(r'Channel damping strength $\gamma$')

ax[0, 1].legend()

ax[0, 1].set_title('(b) 4-party QSSCM')

ax[1, 0].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_list[5], label = r'No encoding $e_1$', color = '#a00')
ax[1, 0].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_lQEC[5], label = r'Repetition code $e_f$', color = '#00a', ls = '--')

ax[1, 0].set_xlabel(r'Channel damping strength $\gamma$')
ax[1, 0].set_ylabel(r'Error on secret $e_1$ or $e_f$')

ax[1, 0].legend()

ax[1, 0].set_title('(c) 5-party QSSCM')

ax[1, 1].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_list[6], label = r'No encoding $e_1$', color = '#a00')
ax[1, 1].plot([0.0, 0.02, 0.04, 0.06, 0.08, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 1.0], damp_err_lQEC[6], label = r'Repetition code $e_f$', color = '#00a', ls = '--')

ax[1, 1].set_xlabel(r'Channel damping strength $\gamma$')

ax[1, 1].legend()

ax[1, 1].set_title('(d) 6-party QSSCM')

tick_params(axis = 'both', which = 'both', direction = 'in')

savefig("rep_damp.png", format="png", bbox_inches="tight")

show()

In [ ]:
from matplotlib.pyplot import rcParams, tick_params
from matplotlib import rc

SIZE = 25
rc('font', size = SIZE)
rc('axes', titlesize=SIZE)
rcParams["figure.figsize"] = (13, 10)
rcParams['font.family'] = 'serif'
rcParams['font.serif'] = ['Arial'] + rcParams['font.serif']

In [ ]:
from matplotlib.pyplot import figure, xticks, yticks, show, savefig, tick_params
from numpy import linspace, meshgrid, sqrt

x = y = z = linspace(0, 1, 41)
A, B, C = meshgrid(x, y, z)
values = (4+2*(A+B+C)-2*(A*B+B*C+C*A)+2*A*B*C-(1-A)*(1-C)*sqrt(1-B)-(1-B)*sqrt((1-A)*(1-C))-2*sqrt((1-A)*(1-B)*(1-C)))/12

fig = figure()
ax = fig.add_subplot(111, projection='3d')
scatter = ax.scatter(A[::-1], B, C, c = [[x[::-1] for x in y][::-1] for y in values])
cbar = fig.colorbar(scatter, ticks = [0, 0.25, 0.5], location='left', shrink = 0.8, pad = 0)
cbar.ax.set_yticklabels(['0.0', '0.25', '0.5'], fontsize = 25)
xticks([0, 1])
ax.set_xticklabels(['1.0', '0.0'], fontsize = 25)
yticks([0, 1])
ax.set_yticklabels(['0.0', '1.0'], fontsize = 25)
ax.set_zticks([0, 1])
ax.set_zticklabels(['1.0', '0.0'], ha = 'left', fontsize = 25)

ax.set_xlabel(r'$\gamma_A$', labelpad = 1, fontsize = 25)
ax.set_ylabel(r'$\gamma_B$', labelpad = 1, fontsize = 25)
ax.set_zlabel(r'$\gamma_C$', labelpad = 1, fontsize = 25)

ax.text(-0.35, -0.35, -0.6, r'Error on secret $e_{1,g}^a$', weight = 'bold', fontsize = 20)

tick_params(axis = 'both', which = 'both', direction = 'in')

savefig("err_amp_gen.png", format = "png", bbox_inches = "tight")

show()